1. Test learning rates of 0.001, 0.0003, 0.0001
2. Weight decay of 0, 0.0001, 0.001
3. Batch size of 16, 32, 64

In [ ]:
# import the required libraries and packages
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from copy import deepcopy

# see if cuda is available
print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu name:", torch.cuda.get_device_name(0))

# load dataset in
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

batch_size = 64

train_dir = "../data/train"
test_dir  = "../data/test"

train_dataset = torchvision.datasets.ImageFolder(
    root=train_dir,
    transform=transform
)

test_dataset = torchvision.datasets.ImageFolder(
    root=test_dir,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

testloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

num_classes = len(train_dataset.classes)
num_epochs = 5

learning_rates = [0.001, 0.0003, 0.0001]
weight_decays = [0.0, 1e-4, 1e-3]
batch_sizes   = [16, 32, 64]

best_f1 = -1.0
best_config = None
best_state_dict = None

for lr in learning_rates:
    for wd in weight_decays:
        for bs in batch_sizes:
            print(f"\n=== Training with lr={lr}, weight_decay={wd}, batch_size={bs} ===")

            # data loaders
            trainloader = torch.utils.data.DataLoader(
                train_dataset,
                batch_size=bs,
                shuffle=True,
                num_workers=4,
            )

            testloader = torch.utils.data.DataLoader(
                test_dataset,
                batch_size=bs,
                shuffle=False,
                num_workers=4,
            )

            # model for hyper parameters
            weights = models.ResNet18_Weights.IMAGENET1K_V1
            model = models.resnet18(weights=weights)

            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, num_classes)

            model = model.to(device)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(
                model.parameters(),
                lr=lr,
                momentum=0.9,
                weight_decay=wd,
            )

            for epoch in range(num_epochs):
                # train
                model.train()
                running_loss = 0.0
                total = 0

                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    total += labels.size(0)

                epoch_loss = running_loss / total

                # evaluate
                model.eval()
                tp = fp = fn = tn = 0

                with torch.no_grad():
                    for inputs, labels in testloader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)

                        # 1 = rotten, 0 = fresh
                        tp += ((preds == 1) & (labels == 1)).sum().item()
                        fp += ((preds == 1) & (labels == 0)).sum().item()
                        fn += ((preds == 0) & (labels == 1)).sum().item()
                        tn += ((preds == 0) & (labels == 0)).sum().item()

                precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
                recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
                f1        = (
                    2 * precision * recall / (precision + recall)
                    if (precision + recall) > 0 else 0.0
                )

                print(
                    f"lr={lr}, wd={wd}, bs={bs} | "
                    f"Epoch {epoch+1}/{num_epochs} | "
                    f"Loss: {epoch_loss:.4f} | "
                    f"F1: {f1:.4f} (P: {precision:.4f}, R: {recall:.4f})"
                )

                # keep track of best model
                if f1 > best_f1:
                    best_f1 = f1
                    best_config = {
                        "lr": lr,
                        "weight_decay": wd,
                        "batch_size": bs,
                        "epoch": epoch + 1,
                    }
                    best_state_dict = deepcopy(model.state_dict())

# save the best model
os.makedirs("results", exist_ok=True)
best_model_path = "../results/resnet_best_grid.pt"
torch.save(best_state_dict, best_model_path)

print("\n=== Best configuration ===")
print(best_config)
print(f"Best F1: {best_f1:.4f}")
print(f"Saved best model to: {best_model_path}")



torch version: 2.9.0+cu126
cuda available: True
gpu name: NVIDIA GeForce GTX 1650
Using device: cuda:0

=== Training with lr=0.001, weight_decay=0.0, batch_size=16 ===
lr=0.001, wd=0.0, bs=16 | Epoch 1/5 | Loss: 0.0927 | F1: 0.9984 (P: 1.0000, R: 0.9967)
lr=0.001, wd=0.0, bs=16 | Epoch 2/5 | Loss: 0.0171 | F1: 0.9993 (P: 1.0000, R: 0.9987)
lr=0.001, wd=0.0, bs=16 | Epoch 3/5 | Loss: 0.0058 | F1: 0.9987 (P: 1.0000, R: 0.9974)
lr=0.001, wd=0.0, bs=16 | Epoch 4/5 | Loss: 0.0082 | F1: 0.9993 (P: 0.9993, R: 0.9993)
lr=0.001, wd=0.0, bs=16 | Epoch 5/5 | Loss: 0.0052 | F1: 1.0000 (P: 1.0000, R: 1.0000)

=== Training with lr=0.001, weight_decay=0.0, batch_size=32 ===
lr=0.001, wd=0.0, bs=32 | Epoch 1/5 | Loss: 0.0935 | F1: 0.9967 (P: 0.9980, R: 0.9954)
lr=0.001, wd=0.0, bs=32 | Epoch 2/5 | Loss: 0.0179 | F1: 0.9993 (P: 1.0000, R: 0.9987)
lr=0.001, wd=0.0, bs=32 | Epoch 3/5 | Loss: 0.0073 | F1: 0.9987 (P: 0.9993, R: 0.9980)
lr=0.001, wd=0.0, bs=32 | Epoch 4/5 | Loss: 0.0067 | F1: 1.0000 (P: 1.0